### Exercicio 1 - Inteligencia Artificial



##### Adicionando dependencias e importações

In [14]:
import pandas as pd
import requests 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import OneHotEncoder
from requests.packages.urllib3.exceptions import InsecureRequestWarning


#### Coletando dados

In [15]:
url = "https://barramento.ifg.edu.br/ifg_barramento_ws/ws/252"
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

column_names = [
    "Campus",
    "Ano de Ingresso",
    "Período Letivo",
    "Curso",
    "Modalidade",
    "Presencial/Distância",
    "Sexo",
    "Nível",
    "Renda per Capita",
    "Etnia",
    "Ano de Nascimento",
    "Situação"
]

try:
    response = requests.get(url, verify=False)
    response.raise_for_status()

    data = response.json()

    processed_data = []
    for item in data:
        row = [sub_item["valor"] for sub_item in item["itens"]]
        processed_data.append(row)

    df = pd.DataFrame(processed_data, columns=column_names)

    output_path = 'data/data.csv'
    df.to_csv(output_path, index=False)
except requests.exceptions.RequestException as e:
    print(f"Erro na requisição: {e}")
except ValueError as e:
    print(f"Erro ao processar os dados: {e}")

#### Pré-processando os dados

In [16]:
def pre_process(df):
    """
    Trata valores nulos em um DataFrame automaticamente:
    - Numéricos: preenche com a média.
    - Categóricos: preenche com 'Desconhecido'.
    - Se a coluna tiver apenas dois valores únicos, substitui por 0 e 1.
    """
    for coluna in df.columns:
        if df[coluna].dtype in ["float64", "int64"]:  
            df[coluna].fillna(df[coluna].mean(), inplace=True)
        elif df[coluna].dtype == "object":
            if df[coluna].nunique() == 2:
                df[coluna] = df[coluna].map({df[coluna].unique()[0]: 0, df[coluna].unique()[1]: 1}).fillna(df[coluna])
            else:
                df[coluna].fillna("Desconhecido", inplace=True)
    return df

df = pre_process(df)

C:\Users\felip\AppData\Local\Temp\ipykernel_88300\1323206550.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[coluna].fillna("Desconhecido", inplace=True)


#### Pós-processamento e transformação

In [ ]:
def onehotencoder(df, colunas_categoricas):
    """
    Aplica OneHotEncoding às colunas categóricas especificadas no DataFrame.
    
    Parâmetros:
        df (pd.DataFrame): O DataFrame original.
        colunas_categoricas (list): Lista de nomes de colunas categóricas a serem transformadas.
        
    Retorno:
        pd.DataFrame: Um novo DataFrame com as colunas categóricas codificadas e as demais mantidas.
    """
    
    encoder = OneHotEncoder(sparse=False, drop="first")  
    
    encoded_arrays = encoder.fit_transform(df[colunas_categoricas])
    
    encoded_df = pd.DataFrame(
        encoded_arrays,
        columns=encoder.get_feature_names_out(colunas_categoricas),
        index=df.index
    )
    
    df = df.drop(columns=colunas_categoricas)
    df = pd.concat([df, encoded_df], axis=1)
    
    return df

colunas_categoricas = []
# df = onehotencoder(df, colunas_categoricas)

TypeError: OneHotEncoder.__init__() got an unexpected keyword argument 'sparse'

#### Aplicação de modelos de regressão

#### Plots e insights